# Installations

In [1]:
!pip install transformers datasets scikit-learn evaluate tqdm kaggle fairscale huggingface_hub -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
ERROR: pip's depende

# ***Fine-Tuning***

# Step 1: Setup
Get environment variables & secrets, set configuration, download data and model artifacts.

In [1]:
from google.colab import userdata
import os

from huggingface_hub import login
import kagglehub

from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import numpy as np

import torch

from transformers import (
    TrainerCallback,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

import evaluate


from functools import partial

import random

In [2]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USR')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

login(token = os.getenv('HF_TOKEN'))
data_path = kagglehub.dataset_download("zynicide/wine-reviews")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Define Configuration

In [3]:
train_bsz, val_bsz = 32, 32 #batch sizes for training and validation
lr = 4e-5 #learning rate
betas = (0.9, 0.98) #for Adam optimizer
n_epochs = 5 #training rounds
eps = 1e-6 #to prevent division by 0
warm_up_ratio = 0.1 #gradual learning rate increase from 0 to target LR during first 10% of training steps, helps stabilize early training by preventing large gradient updates with random initialization
max_grad_norm = 1.0 #prevent exploding gradients

params = {}

params['train_bsz'] = train_bsz
params['val_bsz'] = val_bsz
params['lr'] = lr
params['betas'] = betas
params['n_epochs'] = n_epochs
params['eps'] = eps
params['warm_up_ratio'] = warm_up_ratio
params['max_grad_norm'] = max_grad_norm

class CFG:
  file_1_path = f"{data_path}/winemag-data_first150k.csv"
  file_2_path = f"{data_path}/winemag-data-130k-v2.csv"
  columns = [
    'country',
    'description',
    'designation',
    'points',
    'price',
    'province',
    'region_1',
    'region_2',
    'variety',
    'winery'
    ]

  random_state = 1

  id2label = {0: 'bad', 1: 'average', 2: 'good', 3: 'excellent'}
  label2id = {'bad': 0, 'average': 1, 'good': 2, 'excellent': 3}
  num_labels = len(id2label)

  checkpoint = 'answerdotai/ModernBERT-base'

  hf_repository_id = 'ModernBERT_wine_quality_reviews_ft'

  params = params

  output_dir = os.path.join('output_data', 'ModernBERT_wine_quality_reviews_ft')
  output_data_path = 'output_data'

In [4]:
config = CFG()

## Define some functions

In [5]:
"""
def preprocessing(batch, tokenizer):
    labels = [x for x in batch['label']]
    token_dict = tokenizer(batch['text'], truncation = False)
    return dict({'input_ids': token_dict.input_ids, 'label':labels})
"""
#augmentation
def preprocessing(batch, tokenizer, drop_prob=0.05):
    texts = [x for x in batch['text']]

    # Apply random word dropping during training
    augmented_texts = []
    for text in texts:
        words = text.split()
        kept_words = [word for word in words if random.random() > drop_prob]
        augmented_texts.append(' '.join(kept_words))

    labels = [x for x in batch['label']]
    token_dict = tokenizer(augmented_texts, truncation=False)
    return dict({'input_ids': token_dict.input_ids, 'label':labels})

In [6]:
def bin_label(x):
    if x <= 85:
        return 0
    if x > 85 and x <= 88:
        return 1
    if x > 88 and x <= 92:
        return 2
    if x > 92:
        return 3

In [7]:
class MetricsCallback(TrainerCallback):
  def __init__(self):
    self.training_history = {"train": [], "eval": []}

  def on_log(self, args, state, control, logs=None, **kwargs):
    if logs is not None:
      if "loss" in logs:  # Training logs
        self.training_history["train"].append(logs)
      elif "eval_loss" in logs:  # Evaluation logs
        self.training_history["eval"].append(logs)

In [8]:
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")
    load_f1 = evaluate.load("f1")

    logits, labels = eval_pred
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
    predictions = np.argmax(logits, axis=-1)

    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]

    return {"accuracy": accuracy, "f1": f1}

## Download model artifacts

In [9]:
#Define the tokenizer and data collator
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 2: Prepare the data

### Preprocessing

In [10]:
df1 = pd.read_csv(config.file_1_path, usecols=config.columns)
df2 = pd.read_csv(config.file_2_path, usecols=config.columns)

df = pd.concat([df1, df2], ignore_index=True)

df = df.drop_duplicates()

df['label'] = df['points'].apply(bin_label)
df['text'] = 'Country: ' + df['country'] + '; ' + 'Variety: ' + df['variety'] + '; ' + 'Description: ' + df['description']

df = df[['text', 'label']].drop_duplicates()

df = df.dropna()

<ipython-input-10-ecdb51dca7ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['points'].apply(bin_label)


In [11]:
df.head()

,text,label
0,Country: US; Variety: Cabernet Sauvignon; Desc...,3
1,Country: Spain; Variety: Tinta de Toro; Descri...,3
2,Country: US; Variety: Sauvignon Blanc; Descrip...,3
3,Country: US; Variety: Pinot Noir; Description:...,3
4,Country: France; Variety: Provence red blend; ...,3


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169401 entries, 0 to 280900
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    169401 non-null  object
 1   label   169401 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ MB


In [13]:
100*(df['label'].value_counts()/len(df))

,count
label,
2,35.563544
1,35.414195
0,19.645693
3,9.376568


#### Sanity check

In [14]:
num_labels = len(df['label'].unique())
assert num_labels == len(config.id2label), "The number of labels does not match the number of unique labels in the dataset"

### Splitting

In [15]:
dataset = Dataset.from_pandas(df)

preprocessing = partial(preprocessing, tokenizer=tokenizer)

dataset = dataset.map(preprocessing, batched=True)

dataset = dataset.train_test_split(test_size=0.2, seed=config.random_state)

Map:   0%|          | 0/169401 [00:00<?, ? examples/s]

# Step 3: Setup Training Specs

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=config.num_labels,
    id2label=config.id2label,
    label2id=config.label2id,
    )

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
     #Basic training parameters
     output_dir=config.output_dir, #temporary storage for checkpoints before pushing to hub
     learning_rate=config.params['lr'],
     per_device_train_batch_size=config.params['train_bsz'],
     per_device_eval_batch_size=config.params['val_bsz'],
     num_train_epochs=config.params['n_epochs'],
     adam_beta1=config.params['betas'][0],
     adam_beta2=config.params['betas'][1],
     adam_epsilon=config.params['eps'],
     warmup_ratio=config.params['warm_up_ratio'],
     max_grad_norm=config.params['max_grad_norm'],

     # Logging and evaluation parameters
     logging_strategy="steps",  # Log based on steps
     logging_steps=350,  # Log every 350 steps
     evaluation_strategy="steps",
     save_steps=350,

     # Hub integration parameters
     report_to="tensorboard",  # Enable TensorBoard reporting
     push_to_hub=True,
     hub_strategy="every_save",  # Push whenever we save
     hub_model_id=config.hf_repository_id,  # Where to push the model

     # Additional useful parameters for monitoring
     metric_for_best_model="eval_loss",  # Track best model based on eval loss
     load_best_model_at_end=True,  # Load best model when training ends
     greater_is_better=False,  # For loss, lower is better
     )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )

metrics_callback = MetricsCallback()
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=7, #every early_stopping_patience * logging_steps
    early_stopping_threshold=0.005 #early_stopping_threshold-percent improvement at least
)

trainer.add_callback(metrics_callback)
trainer.add_callback(early_stopping_callback)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Step 4: Run fine-tuning, push to hub & analyze results

In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
350,1.177500,0.999267,0.533632,0.522519
700,0.949600,0.963167,0.556861,0.541120
1050,0.865800,0.825659,0.627254,0.627177
1400,0.828700,0.865519,0.599835,0.574215
1750,0.801800,0.763790,0.658009,0.651023
2100,0.791500,0.748073,0.666067,0.666228
2450,0.767000,0.757203,0.662584,0.661286
2800,0.752500,0.722276,0.674685,0.671907
3150,0.749800,0.721593,0.676810,0.671187
3500,0.725800,0.717265,0.676220,0.673259


Step,Training Loss,Validation Loss,Accuracy,F1
350,1.177500,0.999267,0.533632,0.522519
700,0.949600,0.963167,0.556861,0.541120
1050,0.865800,0.825659,0.627254,0.627177
1400,0.828700,0.865519,0.599835,0.574215
1750,0.801800,0.763790,0.658009,0.651023
2100,0.791500,0.748073,0.666067,0.666228
2450,0.767000,0.757203,0.662584,0.661286
2800,0.752500,0.722276,0.674685,0.671907
3150,0.749800,0.721593,0.676810,0.671187
3500,0.725800,0.717265,0.676220,0.673259


TrainOutput(global_step=9800, training_loss=0.6810104307836415, metrics={'train_runtime': 3770.3345, 'train_samples_per_second': 179.719, 'train_steps_per_second': 5.616, 'total_flos': 2.272931072003251e+16, 'train_loss': 0.6810104307836415, 'epoch': 2.3140495867768593})

In [19]:
trainer.evaluate()

{'eval_loss': 0.679003119468689,
 'eval_accuracy': 0.6965851066969688,
 'eval_f1': 0.6966959575345155,
 'eval_runtime': 65.6729,
 'eval_samples_per_second': 515.906,
 'eval_steps_per_second': 16.125,
 'epoch': 2.3140495867768593}

In [20]:
trainer.create_model_card()
trainer.push_to_hub()

events.out.tfevents.1737480496.f3acabd62f42.52016.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/scbtm/ModernBERT_wine_quality_reviews_ft/commit/2d3d83655b50a7104e3cc968cf218d1cc72b773b', commit_message='End of training', commit_description='', oid='2d3d83655b50a7104e3cc968cf218d1cc72b773b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/scbtm/ModernBERT_wine_quality_reviews_ft', endpoint='https://huggingface.co', repo_type='model', repo_id='scbtm/ModernBERT_wine_quality_reviews_ft'), pr_revision=None, pr_num=None)

In [21]:
train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
train_history_df = train_history_df.add_prefix("train_")
eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

In [22]:
train_res_df

,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,1.1775,9.932329,0.000007,0.082645,0.999267,0.533632,0.522519,NaN,NaN,NaN,NaN
1,0.9496,5.989299,0.000013,0.165289,0.963167,0.556861,0.541120,NaN,NaN,NaN,NaN
2,0.8658,4.995427,0.000020,0.247934,0.825659,0.627254,0.627177,NaN,NaN,NaN,NaN
3,0.8287,4.084484,0.000026,0.330579,0.865519,0.599835,0.574215,NaN,NaN,NaN,NaN
4,0.8018,4.185117,0.000033,0.413223,0.763790,0.658009,0.651023,NaN,NaN,NaN,NaN
5,0.7915,4.339653,0.000040,0.495868,0.748073,0.666067,0.666228,NaN,NaN,NaN,NaN
6,0.7670,4.296793,0.000039,0.578512,0.757203,0.662584,0.661286,NaN,NaN,NaN,NaN
7,0.7525,2.468289,0.000039,0.661157,0.722276,0.674685,0.671907,NaN,NaN,NaN,NaN
8,0.7498,4.913958,0.000038,0.743802,0.721593,0.676810,0.671187,NaN,NaN,NaN,NaN
9,0.7258,3.382503,0.000037,0.826446,0.717265,0.676220,0.673259,NaN,NaN,NaN,NaN
